In [ ]:
!pip uninstall tensorflow -y

Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0


In [ ]:
!pip install tensorflow==2.15.0

  Using cached tensorflow-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
Using cached tensorflow-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (475.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.15.0 which is incompatible.


In [ ]:
import tensorflow
print(tensorflow.__version__)

2.15.0


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/labeled_twitter_CNN_km2_Exp-10w_remove-7-words_SHAP-LIME_improve.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123149 entries, 0 to 123148
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   text     123149 non-null  object
 1   cluster  123149 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.9+ MB


In [ ]:
print(df.isnull().sum().sum())
print(df.isna().sum().sum())
print(df.duplicated().sum().sum())

0
0
0


# CNN Classification Model

In [ ]:
from sklearn.preprocessing import LabelEncoder

X = df['text']
encoder = LabelEncoder()
y = encoder.fit_transform(df['cluster'])
print("shape of input data: ", X.shape)
print("shape of target variable: ", y.shape)

shape of input data:  (123149,)
shape of target variable:  (123149,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers

max_words = 5000
max_len = 200
embedding_dim = 128

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')
vocab_size = len(word_index) + 1

X_train_tweets = pad_sequences(X_train_sequences, maxlen=max_len)
X_test_tweets = pad_sequences(X_test_sequences, maxlen=max_len)

Found 77196 unique tokens.


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

oversample = SMOTE(sampling_strategy=1, random_state=42)
over_X_train, over_y_train = oversample.fit_resample(X_train_tweets, y_train)
#over_X_train, over_X_test, over_y_train, over_y_test = train_test_split(over_X, over_y, test_size=0.20, random_state=42, stratify=over_y)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


In [ ]:
from collections import Counter

print('Original dataset shape {}'.format(Counter(y_train)))
print('Resampled dataset shape {}'.format(Counter(over_y_train)))

Original dataset shape Counter({0: 55965, 1: 42554})
Resampled dataset shape Counter({1: 55965, 0: 55965})


In [ ]:
print(np.count_nonzero(over_y_train == 0))
print(np.count_nonzero(over_y_train == 1))

print(np.count_nonzero(y_test == 0))
print(np.count_nonzero(y_test == 1))

55965
55965
13991
10639


In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# matthews_correlation_coefficient
def mcc_m(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))

    num = tp * tn - fp * fn
    den = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    return num / K.sqrt(den + K.epsilon())

def balanced_acc_m(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))

    sensitivity = tp / (tp + fn)
    specifity = tn / (fp + tn)
    return (sensitivity + specifity) / (2 + K.epsilon())

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, GlobalMaxPooling1D, Dropout
from keras.callbacks import ModelCheckpoint

In [ ]:
import keras
keras.__version__

'2.15.0'

In [ ]:
model = Sequential([
  Embedding(vocab_size, embedding_dim, input_length=max_len),
  Conv1D(64, 5, activation='tanh'),
  GlobalMaxPooling1D(),
  Flatten(),
  Dropout(0.5),
  Dense(32, activation='tanh'),
  Dense(8, activation='tanh'),
  Dense(2, activation='tanh'),
  Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.1))
])

#model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy', metrics=['accuracy', balanced_acc_m, precision_m, recall_m, f1_m, mcc_m, keras.metrics.AUC()])
checkpoint = ModelCheckpoint("best_epoch_cnn_model.h5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)
history = model.fit(over_X_train, over_y_train, batch_size=64, epochs=50, validation_data=(X_test_tweets, y_test), callbacks=[checkpoint, stop_early])

Epoch 1/50
1749/1749 [==============================] - ETA: 0s - loss: 0.6052 - accuracy: 0.7683 - balanced_acc_m: 0.7682 - precision_m: 0.7784 - recall_m: 0.7546 - f1_m: 0.7627 - mcc_m: 0.5377 - auc: 0.8298
Epoch 1: val_accuracy improved from -inf to 0.81161, saving model to best_epoch_cnn_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1749/1749 [==============================] - 43s 20ms/step - loss: 0.6052 - accuracy: 0.7683 - balanced_acc_m: 0.7682 - precision_m: 0.7784 - recall_m: 0.7546 - f1_m: 0.7627 - mcc_m: 0.5377 - auc: 0.8298 - val_loss: 0.5452 - val_accuracy: 0.8116 - val_balanced_acc_m: 0.8085 - val_precision_m: 0.7808 - val_recall_m: 0.7844 - val_f1_m: 0.7794 - val_mcc_m: 0.6164 - val_auc: 0.8685
Epoch 2/50
1744/1749 [============================>.] - ETA: 0s - loss: 0.5522 - accuracy: 0.8016 - balanced_acc_m: 0.8016 - precision_m: 0.8094 - recall_m: 0.7907 - f1_m: 0.7970 - mcc_m: 0.6039 - auc: 0.8586
Epoch 2: val_accuracy did not improve from 0.81161
1749/1749 [==============================] - 20s 12ms/step - loss: 0.5522 - accuracy: 0.8017 - balanced_acc_m: 0.8016 - precision_m: 0.8095 - recall_m: 0.7906 - f1_m: 0.7970 - mcc_m: 0.6040 - auc: 0.8586 - val_loss: 0.5452 - val_accuracy: 0.8100 - val_balanced_acc_m: 0.8082 - val_precision_m: 0.7732 - val_recall_m: 0.7935 - val_f1_m: 0.7803 - val_mcc_m: 0.6

In [ ]:
import json

results_cnn = json.dumps(history.history)

with open('results_cnn_full_clean_smote-train_depressive.json', 'w') as archivo:
    archivo.write(results_cnn)